# Label simulation snapshots

Every time we run a simulation, we will output several snapshots. Probably a number around 10.

Initially the snapshots will be labeled by the output redshift and the cosmology in the box, but in this notebook we show how to relabel the snapshot using the variables that will be used by the emulator.

Note that the emulator does not need to use the same parameters than the likelihood. While the likelihood parameterizes the linear power in velocity units, for better comparison with the data, from the point of view of the emulator is more convenient to use the linear power in comoving separations. This means that the emulator will return the flux 1D power spectrum in comoving units as well, and it means that we need to use comoving separations for the other relevant scales: temperature and pressure smoothing. 

We discuss these issues in this notebook.

In [1]:
%matplotlib inline
import numpy as np
## Set default plot size, as normally its a bit too small
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['savefig.dpi'] = 120
mpl.rcParams['figure.dpi'] = 120
import camb
from camb import model, initialpower
import fit_pk
import read_genic

### Setup cosmological model in the box

In [2]:
# read cosmology from GenIC file
cosmo_genic = 'param_keir.genic'
cosmo_dict = read_genic.camb_from_genic(cosmo_genic)
print(cosmo_dict)
# obtain CAMB object 
pars = fit_pk.get_cosmology(cosmo_dict)

{'H0': 74.33999999999999, 'omch2': 0.10600731605200002, 'ombh2': 0.026692683947999996, 'mnu': 0.0, 'omk': 5.551115123125783e-17, 'TCMB': 2.7255, 'As': 2.86213e-09, 'ns': 0.906429}


### Read list of relevant snapshots

In [3]:
# should read these from Gadget parameter file
snap_zs=[2.0,3.0,4.0]
Nsnap=len(snap_zs)

### Compute linear power at each snapshot

In [4]:
k_Mpc, zs_out, linP_Mpc=fit_pk.get_linP_Mpc(pars,snap_zs)
# make sure that the order did not change (in spite of warning)
np.testing.assert_allclose(snap_zs,zs_out)
print(min(k_Mpc),'< k_Mpc <',max(k_Mpc))

Note: redshifts have been re-sorted (earliest first)
7.434e-05 < k_Mpc < 14.868


### Relabel each snapshot in simulation

In [5]:
def fit_P_Mpc(k_Mpc,P_Mpc,kp_Mpc=0.7,deg=2):
    kmin_Mpc=kp_Mpc*0.8
    kmax_Mpc=kp_Mpc/0.8
    P_fit=fit_pk.fit_polynomial(kmin_Mpc/kp_Mpc,kmax_Mpc/kp_Mpc,k_Mpc/kp_Mpc,P_Mpc,deg=deg)
    return P_fit

In [6]:
def fit_astro_params(snap,z,pars):
    # get mean flux from box (for now just compute toy model)
    mf=np.exp((np.log(0.8))*(((1+z)/3.25)**3.2))
    # measure pressure smoothing length from box (in Mpc)
    kF_Mpc=10.0
    # measure TDR from box
    T_0=1.5e4
    gamma=1.6
    # compute thermal broadening length, in km/s
    sigma_T0_kms=9.1*np.sqrt(T_0/1.e4)
    # translate length to Mpc
    dkms_dhMpc=fit_pk.dkms_dhMpc(pars,z)
    h=pars.H0/100.0
    sigma_T0_Mpc=sigma_T0_kms/dkms_dhMpc/h
    astro_params={'mf':mf, 'kF_Mpc':kF_Mpc, 'gamma':gamma, 'sigma_T0_Mpc':sigma_T0_Mpc}
    return astro_params

In [7]:
def measure_P1D_snapshot(snap):
    k1D_Mpc=np.linspace(0.1,10.0,100)
    P1D_Mpc=100.0*np.ones_like(k1D_Mpc)
    return k1D_Mpc,P1D_Mpc

In [8]:
# will store information about snapshots
snapshots=[]
for snap in range(Nsnap):
    z=snap_zs[snap]
    # fit linear power around kp_Mpc=0.7
    P_Mpc=linP_Mpc[snap]
    P_fit=fit_P_Mpc(k_Mpc,P_Mpc)
    print(z,np.array(P_fit))
    # measure astro parameters from snapshot
    astro_params = fit_astro_params(snap,z,pars)
    print(z,astro_params)
    # measure 1D power spectrum from snapshot
    k1D_Mpc, P1D_Mpc= measure_P1D_snapshot(snap)
    snapshot={'astro_params':astro_params,'linP_Mpc':P_fit,'k1D_Mpc':k1D_Mpc,'P1D_Mpc':P1D_Mpc}
    snapshots.append(snapshot)

2.0 [-0.09996443 -2.39232305  3.4395078 ]
2.0 {'mf': 0.84137217213506155, 'kF_Mpc': 10.0, 'gamma': 1.6, 'sigma_T0_Mpc': 0.16711828413965304}
3.0 [-0.09995883 -2.39232365  2.88772469]
3.0 {'mf': 0.64813245871103942, 'kF_Mpc': 10.0, 'gamma': 1.6, 'sigma_T0_Mpc': 0.14932802034277648}
4.0 [-0.09996496 -2.39232347  2.45038825]
4.0 {'mf': 0.41244644004470926, 'kF_Mpc': 10.0, 'gamma': 1.6, 'sigma_T0_Mpc': 0.13512556005345677}


### Feeding snapshots to the emulator

The emulator will have a list of snapshots, identified by a list of parameters $\mu$:
$$ \mu = \{ \mu_P, k_F, \bar F, \sigma_T, \gamma \} $$

where $\mu_P$ is a set of (3) parameters describing the linear power in the snapshot, in units of Mpc.

Note that while each snapshot will correspond to a set of parameters $\{ \mu_P, k_F \}$, we will be able to use the same snapshot to generate spectra with different values of $ \{ \bar F, \sigma_T, \gamma \}$, since these can be changed in post-process.

In [9]:
for snapshot in snapshots:
    astro_params=snapshot['astro_params']
    linP_params=snapshot['linP_Mpc']
    print(astro_params,np.array(linP_params))

{'mf': 0.84137217213506155, 'kF_Mpc': 10.0, 'gamma': 1.6, 'sigma_T0_Mpc': 0.16711828413965304} [-0.09996443 -2.39232305  3.4395078 ]
{'mf': 0.64813245871103942, 'kF_Mpc': 10.0, 'gamma': 1.6, 'sigma_T0_Mpc': 0.14932802034277648} [-0.09995883 -2.39232365  2.88772469]
{'mf': 0.41244644004470926, 'kF_Mpc': 10.0, 'gamma': 1.6, 'sigma_T0_Mpc': 0.13512556005345677} [-0.09996496 -2.39232347  2.45038825]
